# Active Learning dashboard

In [3]:
import json
import psycopg2
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

with open('config.json') as f:
    conf = json.load(f)
conn_str = "host={} dbname={} user={} password={}".format(conf['host'], conf['database'], conf['user'], conf['passw'])
conn = psycopg2.connect(conn_str)
plotly.tools.set_credentials_file(username=conf['plotlyusername'], api_key=conf['plotlyapikey'])

Plotting a pie chart for current image tagging status:

In [4]:
names = []
numbers = []
cursor = conn.cursor()
cursor.execute('select count(a.imageid), b.tagstatename from Image_Tagging_State a join tagstate b ON a.tagstateid = b.tagstateid group by b.tagstatename')
for (number, name) in cursor:
    names.append(name)
    numbers.append(number)

trace = go.Pie(labels=names, values=numbers)
py.iplot([trace], filename='basic')

Plotting a time chart for the tagging activity over the last few days: 

In [5]:
dates = []
numimages = []
cursor = conn.cursor()
cursor.execute("select count(imageid), date_trunc('day', modifieddtim) timerange from image_tagging_state_audit group by date_trunc('day', modifieddtim) order by timerange")
for (numimage, date) in cursor:
    x = pd.to_datetime(date)
    dates.append(x)
    numimages.append(numimage)

# Create a trace
trace = go.Scatter(
    x = dates,
    y = numimages,
    # mode = 'markers'
    name = 'Number of tagging activities'
)
data = [trace]
layout = dict(title = 'Number of tagging activities by date', xaxis=dict(title='Date'), yaxis=dict(title='Tagging activities'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='basic-scatter')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~samiyaakhtar/0 or inside your plot.ly account where it is named 'basic-scatter'
